In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as ss
from retina_to_connectome_funcs import get_synapse_df

In [9]:
from utils import synapses_to_matrix_and_dict


def create_basic_data(side=None):
    synapses = get_synapse_df(side)
    char = "" if side is None else f"{side}_"
    matrix, root_id_to_index = synapses_to_matrix_and_dict(synapses)
    ss.save_npz(f"adult_data/{char}synaptic_matrix.npz", matrix)
    pd.DataFrame(root_id_to_index.items(), columns=["root_id", "index"]).to_csv(
        f"adult_data/{char}root_id_to_index.csv", index=False
    )

In [10]:
# create_basic_data()
create_basic_data("left")
create_basic_data("right")

In [67]:
connections = pd.read_csv("adult_data/connections.csv")

# there are repeated connections, so we add them
connections = connections.groupby(["pre_root_id", "post_root_id"]).sum("syn_count").reset_index()
connections["pre_root_id"] = connections["pre_root_id"].astype("string")
connections["post_root_id"] = connections["post_root_id"].astype("string")

In [116]:
cl = pd.read_csv("adult_data/classification.csv")
cl["root_id"] = cl["root_id"].astype("string")
cl = cl[cl["side"] == "right"]

In [117]:
# remove duplicated cell_types, but leaving the ones we prefer
cl["priority"] = np.where(cl["cell_type"] == "R8", 1, 
                    np.where(cl["cell_type"] == "R7", 2,
                        np.where(cl["cell_type"] == "R1-6", 3,
                            np.where(cl["cell_type"].notnull(), 4, 5))))
cl.sort_values(by=["root_id", "priority"], inplace=True)
cl = cl.drop_duplicates(subset="root_id", keep="first")
cl.drop(columns="priority", inplace=True)

In [118]:
cl_connections = connections.merge(cl, left_on="pre_root_id", right_on="root_id")
cl_connections = cl_connections.merge(cl, left_on="post_root_id", right_on="root_id", suffixes=("_pre", "_post"))

In [119]:
# Extract unique identifiers from both dataframes
unique_cl_ids = set(cl["root_id"])
unique_pre_ids = set(cl_connections["pre_root_id"])  # Pre-synaptic identifiers
unique_post_ids = set(cl_connections["post_root_id"])  # Post-synaptic identifiers

In [120]:
only_in_cl = (unique_cl_ids - (unique_pre_ids | unique_post_ids))

In [121]:
# leave cl not in only_in_cl
filtered_cl = cl[~cl["root_id"].isin(only_in_cl)]


In [ ]:

# Determine the common identifiers
# Neurons that are in 'cl' and either 'pre_root_id' or 'post_root_id' in 'connections'
common_ids = unique_cl_ids & (unique_pre_ids | unique_post_ids)

# Filter the 'cl' dataframe to include only rows with 'root_id' in common_ids
filtered_cl = cl[cl["root_id"].isin(common_ids)]

# Filter the 'connections' dataframe to include only rows with 'pre_root_id' or 'post_root_id' in common_ids
filtered_connections = connections[
    (connections["pre_root_id"].isin(common_ids))
    | (connections["post_root_id"].isin(common_ids))
]

# The 'filtered_cl' and 'filtered_connections' dataframes now contain only the rows with neurons that match the criteria.

In [2]:
import pandas as pd
cl_connections = pd.read_csv("adult_data/right_connections.csv")
filtered_cl = pd.read_csv("adult_data/right_classification.csv")

In [3]:
# Convert to sets for faster operations
pre_ids_set = set(cl_connections["pre_root_id"].values)
post_ids_set = set(cl_connections["post_root_id"].values)
cl_ids_set = set(filtered_cl["root_id"].values)

# Find neurons not in either column using set operations
missing_both = cl_ids_set - (pre_ids_set | post_ids_set)  # not in either
missing_either = cl_ids_set - (pre_ids_set & post_ids_set)  # not in both
missing_pre = pre_ids_set - cl_ids_set  # in pre but not in cl
missing_post = post_ids_set - cl_ids_set  # in post but not in cl

# Determine lengths as needed
len_missing_both = len(missing_both)
len_missing_either = len(missing_either)

In [6]:
cl_connections

,pre_root_id,post_root_id,syn_count,cell_type.x,cell_type.y
0,720575940596125868,720575940608552405,5,T5c,Tlp6
1,720575940596125868,720575940613059993,5,T5c,NaN
2,720575940596125868,720575940616986553,5,T5c,Y3
3,720575940596125868,720575940618079956,5,T5c,NaN
4,720575940596125868,720575940620124326,8,T5c,Tlp4
...,...,...,...,...,...
1231187,720575940661333889,720575940629764906,5,CB2303,CB2537
1231188,720575940661333889,720575940634776511,5,CB2303,CB2535
1231189,720575940661333889,720575940637151807,12,CB2303,CB2367
1231190,720575940661333889,720575940638996413,7,CB2303,CB2367


In [139]:
# Map neuron root_ids to matrix indices
root_id_to_index = {root_id: index for index, root_id in enumerate(cl_ids_set)}

# Convert root_ids in filtered_synapse_df to matrix indices
pre_indices = cl_connections["pre_root_id"].map(root_id_to_index).values
post_indices = cl_connections["post_root_id"].map(root_id_to_index).values

# Use syn_count as the data for the non-zero elements of the matrix
data = cl_connections["syn_count"].values

# Create the sparse matrix
matrix = ss.coo_matrix(
    (data, (pre_indices, post_indices)),
    shape=(len(cl_ids_set), len(cl_ids_set)),
    dtype=np.int64,
)

In [140]:
pd.DataFrame(root_id_to_index.items(), columns=["root_id", "index"]).to_csv(
    "adult_data/right_root_id_to_index.csv", index=False
)

In [141]:
ss.save_npz("adult_data/right_synaptic_matrix.npz", matrix)

In [145]:
visual = filtered_cl[filtered_cl["cell_type"].isin(["R8", "R7", "R1-6"])]

In [172]:
coords = pd.read_csv(
    "adult_data/coordinates_sep.csv",
    dtype={"root_id": "string"},
    )

In [174]:
# Filter the 'coords' dataframe to include only rows with 'root_id' in common_ids
filtered_coords = coords[coords["root_id"].isin(common_ids)]

In [175]:
filtered_coords

,root_id,x,y,z,supervoxel_id
156,720575940638164864,689560,301620,74920,8.247684e+16
307,720575940630892544,788036,313228,97480,8.416583e+16
313,720575940629327872,723084,188368,139880,8.303787e+16
401,720575940628023296,714996,205072,147960,8.289740e+16
544,720575940630799104,494852,107572,45240,7.909584e+16
...,...,...,...,...,...
238687,720575940631190016,510136,218256,196440,7.937917e+16
238691,720575940631190016,532392,198908,188080,7.980104e+16
238692,720575940631190016,533616,177480,177880,7.980070e+16
238745,720575940640150912,283592,364620,243040,7.551137e+16


In [8]:
from scipy.sparse import load_npz
synaptic_matrix = load_npz("adult_data/good_synaptic_matrix.npz")

In [10]:
import numpy as np
np.log1p(synaptic_matrix.data)

array([1.79175947, 1.79175947, 1.79175947, ..., 1.79175947, 3.04452244,
       1.94591015])

In [1]:
import torch
torch.cuda.is_available()

True